In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
def direction(val):
    val1 = Conversion().convert_int256_bytes_to_int(val, signed=False) 
    val2 = Conversion().convert_int256_bytes_to_int(val, signed=True)
    return val1 != val2

In [3]:
chain = Net.POLYGON
platform = Platform.UNIV3
contract = JSONContract.UniswapV3Pool

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoad().get_contract(web3, abi_path)
swap_filt = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 10

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=swap_filt)

In [8]:
from eth_defi.token import TokenDetails, fetch_erc20_details

In [9]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    args  = Conversion().decode_data(evt["data"])
    token0_address = Conversion().convert_uint256_hex_string_to_address(topics[1])
    token1_address = Conversion().convert_uint256_hex_string_to_address(topics[2])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = token0_address
    event['details']['token1'] = token1_address    
    if(len(args) == 5):
        event['details']['amount0'] = Conversion().convert_int256_bytes_to_int(args[0], signed=direction(args[0])) 
        event['details']['amount1'] = Conversion().convert_int256_bytes_to_int(args[1], signed=direction(args[1])) 
        event['details']['sqrt_price_x96'] = Conversion().convert_int256_bytes_to_int(args[2]) 
        event['details']['liquidity'] = Conversion().convert_int256_bytes_to_int(args[3]) 
        event['details']['tick'] = Conversion().convert_int256_bytes_to_int(args[4], signed=direction(args[4])) 
    
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:61,383,646 tx:0xdf508055da2f20354bab74b62591cd0a2a505bc41a6c0016b89ed070159fe643
Swap at block:61,383,646 tx:0x69b861e6578a78bac7fd7d8a8763a99959548970aa84555d6ac1d4b83aaf6128
Swap at block:61,383,646 tx:0xbdcfdc140e9fe6063d653df025291f40b176a2b26cab26aa9ab46efb49072776
Swap at block:61,383,646 tx:0xbdcfdc140e9fe6063d653df025291f40b176a2b26cab26aa9ab46efb49072776
Swap at block:61,383,646 tx:0x5cfe5d9e7fd2107b227c95730563c8e0241de2f71e79de517701c53497a2e152
Swap at block:61,383,646 tx:0x8afc3f0058c5af52d6ac51ad9574379e786be4e2424e0b1035ba1ea704ea9e81
Swap at block:61,383,646 tx:0x8afc3f0058c5af52d6ac51ad9574379e786be4e2424e0b1035ba1ea704ea9e81
Swap at block:61,383,646 tx:0x8afc3f0058c5af52d6ac51ad9574379e786be4e2424e0b1035ba1ea704ea9e81
Swap at block:61,383,646 tx:0x8afc3f0058c5af52d6ac51ad9574379e786be4e2424e0b1035ba1ea704ea9e81
Swap at block:61,383,646 tx:0x4b64c982567aa53f04d713fc6b1a0a0ebfeed27862c5793c5f275944767f68d0
Swap at block:61,383,646 tx:0x4b64c982567aa53f04d7

In [10]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv3pool,swap,uniswap_v3,0xeef1a9507b3d505f0062f2be9453981255b503c8,0xdf508055da2f20354bab74b62591cd0a2a505bc41a6c...,61383646,1725384743,{'web3_type': <class 'web3._utils.datatypes.Sw...
1,polygon,uniswapv3pool,swap,uniswap_v3,0xa5cd8351cbf30b531c7b11b0d9d3ff38ea2e280f,0x69b861e6578a78bac7fd7d8a8763a99959548970aa84...,61383646,1725384743,{'web3_type': <class 'web3._utils.datatypes.Sw...
2,polygon,uniswapv3pool,swap,uniswap_v3,0x5645dcb64c059aa11212707fbf4e7f984440a8cf,0xbdcfdc140e9fe6063d653df025291f40b176a2b26cab...,61383646,1725384743,{'web3_type': <class 'web3._utils.datatypes.Sw...
3,polygon,uniswapv3pool,swap,uniswap_v3,0x0a6c4588b7d8bd22cf120283b1fff953420c45f3,0xbdcfdc140e9fe6063d653df025291f40b176a2b26cab...,61383646,1725384743,{'web3_type': <class 'web3._utils.datatypes.Sw...
4,polygon,uniswapv3pool,swap,uniswap_v3,0x86f1d8390222a3691c28938ec7404a1661e618e0,0x5cfe5d9e7fd2107b227c95730563c8e0241de2f71e79...,61383646,1725384743,{'web3_type': <class 'web3._utils.datatypes.Sw...
5,polygon,uniswapv3pool,swap,uniswap_v3,0x22177148e681a6ca5242c9888ace170ee7ec47bd,0x8afc3f0058c5af52d6ac51ad9574379e786be4e2424e...,61383646,1725384743,{'web3_type': <class 'web3._utils.datatypes.Sw...
6,polygon,uniswapv3pool,swap,uniswap_v3,0xa374094527e1673a86de625aa59517c5de346d32,0x8afc3f0058c5af52d6ac51ad9574379e786be4e2424e...,61383646,1725384743,{'web3_type': <class 'web3._utils.datatypes.Sw...
7,polygon,uniswapv3pool,swap,uniswap_v3,0x86f1d8390222a3691c28938ec7404a1661e618e0,0x8afc3f0058c5af52d6ac51ad9574379e786be4e2424e...,61383646,1725384743,{'web3_type': <class 'web3._utils.datatypes.Sw...
8,polygon,uniswapv3pool,swap,uniswap_v3,0x0a28c2f5e0e8463e047c203f00f649812ae67e4f,0x8afc3f0058c5af52d6ac51ad9574379e786be4e2424e...,61383646,1725384743,{'web3_type': <class 'web3._utils.datatypes.Sw...
9,polygon,uniswapv3pool,swap,uniswap_v3,0xab52931301078e2405c3a3ebb86e11ad0dfd2cfd,0x4b64c982567aa53f04d713fc6b1a0a0ebfeed27862c5...,61383646,1725384743,{'web3_type': <class 'web3._utils.datatypes.Sw...


In [11]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'swap',
  'platform': 'uniswap_v3',
  'address': '0xeef1a9507b3d505f0062f2be9453981255b503c8',
  'tx_hash': '0xdf508055da2f20354bab74b62591cd0a2a505bc41a6c0016b89ed070159fe643',
  'blk_num': 61383646,
  'timestamp': 1725384743,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0x40DBd3004a80a4D1dDa27776671720AE16890360',
   'token1': '0xbfa1CBFc7B79115947889ac7Cb25801EE8055dF9',
   'amount0': 229681,
   'amount1': -133070131,
   'sqrt_price_x96': 1907497043411218117953941555284,
   'liquidity': 540552236086,
   'tick': 63627}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv3pool',
  'type': 'swap',
  'platform': 'uniswap_v3',
  'address': '0xa5cd8351cbf30b531c7b11b0d9d3ff38ea2e280f',
  'tx_hash': '0x69b861e6578a78bac7fd7d8a8763a99959548970aa84555d6ac1d4b83aaf6128',
  'blk_num': 61383646,
  'timestamp': 1725384743,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0x40DBd3004a80a4D1